In [9]:
import os
from random import seed, random
from os import makedirs, listdir
from os.path import exists, isdir
import random
import argparse
import subprocess

import torch
from cv2 import imwrite

FRAMES_ONSCREEN = 227.3
FRAME_RATE = 25
FPS = 2
TIME_ONSCREEN = (FRAMES_ONSCREEN / FRAME_RATE) * FPS

In [10]:
def concat_video(src: str):
    files = sorted((f for f in listdir(src) if not f.startswith(".") and f.endswith('.mp4')), key=str.lower)
    print(files)

    exists(f'{src}/video.mp4') and os.remove(f'{src}/video.mp4')
    exists(f'{src}/videos.txt') and os.remove(f'{src}/videos.txt')
    with open(f'{src}/videos.txt', 'w') as txt_file:
        for i, f in enumerate(files):
            txt_file.write(f"file '{src}/{f}'\n")
    subprocess.run(args=[f'ffmpeg', '-f', 'concat', '-safe', '0', '-i', f'{src}/videos.txt', '-c', 'copy', f'{src}/video.mp4'])

In [11]:
def video_to_frames(video_src: str, src: str):
    makedirs(f'{src}/frames', exist_ok=True)
    subprocess.run(args=['ffmpeg', '-i', f'{video_src}', '-vf', f'fps={FPS}', f'{src}/frames/%03d.png'])

In [12]:
def frames_to_dataset(frames_src: str, dest: str, test_split: float):
    cow_num = 0
    seed(0)
    frames = sorted((f for f in listdir(frames_src) if not f.startswith(".") and not isdir(f'{frames_src}/{f}')), key=str.lower)
    for i, frame in enumerate(frames):
        subset = random.random()
        faces = frame_to_faces(frame)
        if i % round(TIME_ONSCREEN) == 0:
            cow_num += 1
            print(f'Cow {cow_num}')
            makedirs(f'{dest}/train/cow{cow_num}', exist_ok=True)
            makedirs(f'{dest}/test/cow{cow_num}', exist_ok=True)
        if subset < 1 - test_split:
            # copy(f'{frames_src}/{frame}', f'{dest}/train/cow{cow_num}')  # Copy image to the appropriate directory
            for img in faces:
                imwrite(img, f'{dest}/train/cow{cow_num}')  # Write cow face to the appropriate directory
        else:
            # copy(f'{frames_src}/{frame}', f'{dest}/test/cow{cow_num}')  # Copy image to the appropriate directory
            for img in faces:
                imwrite(img, f'{dest}/test/cow{cow_num}')  # Write cow face to the appropriate directory

In [13]:
def frame_to_faces(frame):
    # Model
    model = torch.hub.load(repo_or_dir='ultralytics/yolov5', model='custom', path='best.pt')
    for f in os.listdir('/Users/adam/git/agricam/data/video/clean thermal/norm/frames'):
        # Images
        img = f'/Users/adam/git/agricam/data/video/clean thermal/norm/frames/{f}'
        # Inference
        results = model(img)
        # Results
        crops = results.crop()  # or .show(), .save(), .crop(), .pandas(), etc.
        faces = []
        for c in crops:
            faces.append(c.get('im'))

    return faces

In [14]:
# concat_video("/Users/adam/Git/agricam/data/video/clean thermal/norm")
# video_to_frames('/Users/adam/Git/agricam/data/video/clean thermal/norm/video.mp4', '/Users/adam/Git/agricam/data/video/clean thermal/norm')
frames_to_dataset('/Users/adam/Git/agricam/data/video/clean thermal/norm/frames', '/Users/adam/Git/agricam/datasets/20210714', 0.3)

Using cache found in /Users/adam/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-9-4 Python-3.9.13 torch-1.12.1 CPU

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 
Saved 1 image to runs/detect/exp
Saved results to runs/detect/exp

Saved 1 image to runs/detect/exp2
Saved results to runs/detect/exp2

Saved 1 image to runs/detect/exp3
Saved results to runs/detect/exp3

Saved 1 image to runs/detect/exp4
Saved results to runs/detect/exp4

Saved 1 image to runs/detect/exp5
Saved results to runs/detect/exp5

Saved 1 image to runs/detect/exp6
Saved results to runs/detect/exp6

Saved 1 image to runs/detect/exp7
Saved results to runs/detect/exp7

Saved 1 image to runs/detect/exp8
Saved results to runs/detect/exp8

Saved 1 image to runs/detect/exp9
Saved results to runs/detect/exp9

Saved 1 image to runs/detect/exp10
Saved results to runs/detect/exp10

Saved 1 image to runs/detect/exp11
Saved results to runs/detect/exp11



KeyboardInterrupt: 